In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.metrics import f1_score

In [4]:
def load_predictions(dataset_name, partition, fold_number=1):
    path = './Saved_Predict_and_Proba/Folds/' + dataset_name.upper() + '/F' + str(fold_number)  + '/pred_' + partition +'.csv'
    table_pred = pd.read_csv(path)
    if dataset_name.lower() == 'zw':
        label = table_pred['norm']
        preds = table_pred.drop('norm', axis=1)
    else: 
        label = table_pred['class']
        preds = table_pred.drop('class', axis=1)
    preds = preds.drop('Unnamed: 0', axis=1)
    return label, preds

def load_datasets(dataset_name, fold_number):
    label_train, preds_train = load_predictions(dataset_name, 'train', fold_number)
    label_val, preds_val = load_predictions(dataset_name, 'val', fold_number)
    label_test, preds_test = load_predictions(dataset_name, 'test', fold_number)
    return label_train, preds_train, label_test, preds_test, label_val, preds_val

def compile_results(dataset_name):
    algorithms_list = ['SVM', 'MLP', 'KNN', 'RF', 'EXTRA', 'CNN', 'LR', 'NB']
    fe_list = ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']
    results_f1_val = np.zeros((5, len(algorithms_list), len(fe_list)))
    results_f1_test = np.zeros((5, len(algorithms_list), len(fe_list)))

    for fold in range(1, 6):
        _, _, label_test, preds_test, label_val, preds_val = load_datasets(dataset_name, fold)

        for idx_alg, alg in enumerate(algorithms_list):
            for idx_fe, fe in enumerate(fe_list):
                #VAL
                y_pred_val = preds_val.filter(regex=alg+'-'+fe)
                results_f1_val[fold-1, idx_alg, idx_fe] = f1_score(label_val, y_pred_val,average='macro')           
                #TESTE
                y_pred_test = preds_test.filter(regex=alg+'-'+fe)
                results_f1_test[fold-1,idx_alg, idx_fe] = f1_score(label_test, y_pred_test,average='macro')
    return results_f1_val, results_f1_test

algorithms_list = ['SVM', 'MLP', 'KNN', 'RF', 'EXTRA', 'CNN', 'LR', 'NB']
fe_list = ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']

# Getting TD results

In [40]:
dataset_name = 'TD'
results_f1_val, results_f1_test = compile_results('TD')
results_df_test = pd.DataFrame(results_f1_test.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)
results_df_val = pd.DataFrame(results_f1_val.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)

std_df_test = pd.DataFrame(results_f1_test.std(axis=0), columns=fe_list, index=algorithms_list).round(3)
std_df_val = pd.DataFrame(results_f1_val.std(axis=0), columns=fe_list, index=algorithms_list).round(3)

In [41]:
results_df_test.max()

CV       0.715
TFIDF    0.708
W2V      0.689
GLOVE    0.651
FAST     0.694
dtype: float64

In [28]:
std_df_test.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &      CV &   TFIDF &     W2V &   GLOVE &    FAST \\\\\n\\midrule\nSVM   &  0.0015 &  0.0092 &  0.0111 &  0.0052 &  0.0061 \\\\\nMLP   &  0.0181 &  0.0050 &  0.0164 &  0.0086 &  0.0144 \\\\\nKNN   &  0.0251 &  0.0536 &  0.0066 &  0.0071 &  0.0134 \\\\\nRF    &  0.0052 &  0.0081 &  0.0067 &  0.0144 &  0.0083 \\\\\nEXTRA &  0.0067 &  0.0118 &  0.0075 &  0.0055 &  0.0072 \\\\\nCNN   &  0.0435 &  0.0020 &  0.0190 &  0.0358 &  0.0366 \\\\\nLR    &  0.0083 &  0.0077 &  0.0107 &  0.0063 &  0.0045 \\\\\nNB    &  0.0028 &  0.0053 &  0.0048 &  0.0038 &  0.0085 \\\\\n\\bottomrule\n\\end{tabular}\n'

# Getting ZW results

In [29]:
dataset_name = 'ZW'
results_f1_val, results_f1_test = compile_results(dataset_name)

results_df_test = pd.DataFrame(results_f1_test.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)
results_df_val = pd.DataFrame(results_f1_val.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)

std_df_test = pd.DataFrame(results_f1_test.std(axis=0), columns=fe_list, index=algorithms_list).round(3)
std_df_val = pd.DataFrame(results_f1_val.std(axis=0), columns=fe_list, index=algorithms_list).round(3)

In [31]:
results_df_test.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &     CV &  TFIDF &    W2V &  GLOVE &   FAST \\\\\n\\midrule\nSVM   &  0.743 &  0.763 &  0.706 &  0.271 &  0.596 \\\\\nMLP   &  0.763 &  0.714 &  0.707 &  0.543 &  0.720 \\\\\nKNN   &  0.438 &  0.379 &  0.675 &  0.536 &  0.614 \\\\\nRF    &  0.732 &  0.758 &  0.595 &  0.507 &  0.590 \\\\\nEXTRA &  0.734 &  0.753 &  0.573 &  0.474 &  0.558 \\\\\nCNN   &  0.730 &  0.723 &  0.748 &  0.725 &  0.749 \\\\\nLR    &  0.760 &  0.732 &  0.652 &  0.425 &  0.627 \\\\\nNB    &  0.751 &  0.720 &  0.580 &  0.511 &  0.611 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [30]:
std_df_test.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &     CV &  TFIDF &    W2V &  GLOVE &   FAST \\\\\n\\midrule\nSVM   &  0.011 &  0.008 &  0.006 &  0.001 &  0.016 \\\\\nMLP   &  0.009 &  0.009 &  0.011 &  0.012 &  0.009 \\\\\nKNN   &  0.009 &  0.007 &  0.013 &  0.014 &  0.004 \\\\\nRF    &  0.010 &  0.011 &  0.014 &  0.006 &  0.015 \\\\\nEXTRA &  0.011 &  0.010 &  0.015 &  0.007 &  0.019 \\\\\nCNN   &  0.019 &  0.014 &  0.013 &  0.010 &  0.015 \\\\\nLR    &  0.012 &  0.011 &  0.016 &  0.010 &  0.010 \\\\\nNB    &  0.003 &  0.006 &  0.008 &  0.003 &  0.006 \\\\\n\\bottomrule\n\\end{tabular}\n'

# Getting union results (TD+ZW)

In [36]:
dataset_name = 'Union'
results_f1_val, results_f1_test = compile_results(dataset_name)

results_df_test = pd.DataFrame(results_f1_test.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)
results_df_val = pd.DataFrame(results_f1_val.mean(axis=0), columns=fe_list, index=algorithms_list).round(3)

std_df_test = pd.DataFrame(results_f1_test.std(axis=0), columns=fe_list, index=algorithms_list).round(3)
std_df_val = pd.DataFrame(results_f1_val.std(axis=0), columns=fe_list, index=algorithms_list).round(3)

In [38]:
results_df_test

,CV,TFIDF,W2V,GLOVE,FAST
SVM,0.872,0.881,0.758,0.670,0.753
MLP,0.871,0.852,0.757,0.673,0.770
KNN,0.691,0.358,0.666,0.610,0.608
RF,0.866,0.864,0.645,0.644,0.647
EXTRA,0.860,0.855,0.623,0.631,0.608
CNN,0.842,0.815,0.848,0.836,0.855
LR,0.882,0.875,0.724,0.621,0.735
NB,0.850,0.820,0.635,0.604,0.666


In [34]:
results_df_test.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &     CV &  TFIDF &    W2V &  GLOVE &   FAST \\\\\n\\midrule\nSVM   &  0.872 &  0.881 &  0.758 &  0.670 &  0.753 \\\\\nMLP   &  0.871 &  0.852 &  0.757 &  0.673 &  0.770 \\\\\nKNN   &  0.691 &  0.358 &  0.666 &  0.610 &  0.608 \\\\\nRF    &  0.866 &  0.864 &  0.645 &  0.644 &  0.647 \\\\\nEXTRA &  0.860 &  0.855 &  0.623 &  0.631 &  0.608 \\\\\nCNN   &  0.842 &  0.815 &  0.848 &  0.836 &  0.855 \\\\\nLR    &  0.882 &  0.875 &  0.724 &  0.621 &  0.735 \\\\\nNB    &  0.850 &  0.820 &  0.635 &  0.604 &  0.666 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [35]:
std_df_test.to_latex()

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &     CV &  TFIDF &    W2V &  GLOVE &   FAST \\\\\n\\midrule\nSVM   &  0.004 &  0.004 &  0.003 &  0.006 &  0.007 \\\\\nMLP   &  0.004 &  0.004 &  0.005 &  0.009 &  0.003 \\\\\nKNN   &  0.018 &  0.009 &  0.009 &  0.007 &  0.008 \\\\\nRF    &  0.006 &  0.003 &  0.009 &  0.007 &  0.007 \\\\\nEXTRA &  0.004 &  0.003 &  0.010 &  0.007 &  0.008 \\\\\nCNN   &  0.014 &  0.011 &  0.011 &  0.006 &  0.009 \\\\\nLR    &  0.003 &  0.003 &  0.003 &  0.007 &  0.006 \\\\\nNB    &  0.006 &  0.005 &  0.006 &  0.006 &  0.010 \\\\\n\\bottomrule\n\\end{tabular}\n'

# Getting hateval results